In [1]:
import os
import pandas as pd
import numpy as np
import copy
# import hyperloglog
from datasketch import HyperLogLog

# Points
For now i am assuming that primary key is only a single attribute

In [2]:
class Attribute:
    def __init__(self, table_name, attribute_name, values):
        self.table_name = table_name
        self.attribute_name = attribute_name
        self.values = values

        self.uniquness = self.estUniqueness()
        self.cardinality=1
        self.value_length = 1/max(1, max([len(x) for x in values]) - 8)
        self.position = 0
        self.suffix = self.check_suffix()
        
        self.dependent = [] # It's dependent attributes
        self.reference = [] # Attributes that it references from 

        self.pkScore = 0
        self.pkScore += self.uniquness
        self.pkScore += self.cardinality
        self.pkScore += self.value_length
        self.pkScore += self.position
        self.pkScore += self.suffix

    def estUniqueness(self):
        hll = HyperLogLog()
        total = 0
        
        for value in self.values:
            hll.update(str(value).encode('utf8'))
            total +=1
        # print(f"{hll.count()=}")
        
        return hll.count() / total
    
    def check_suffix(self, suffix_list=["key", 'id', 'nr', 'no']):
        for suffix in suffix_list:
            if suffix in self.attribute_name:
                return 1
            else:
                return 0


In [ ]:
def load_csv_files(directory_path):
    attributes = {}

    csv_files = [f for f in os.listdir(directory_path)]

    print(f"Found {len(csv_files)} \n CSV files: {csv_files}")    

    for filename in csv_files:
        file_path = os.path.join(directory_path, filename)
        table_name = os.path.splitext(filename)[0]

        df = pd.read_csv(file_path)
        df = df
        print(f"Processing {filename}: {df.shape[0]} rows, {df.shape[1]} columns")

        for i, column in enumerate(df.columns):
            non_null_values = df[column].to_string().dropna().tolist()
            if non_null_values:
                attr = Attribute(table_name, column, non_null_values)
                attr.position = 1/(i+1)
                attributes[f"{table_name}.{column}"] = attr
                print(f"Added attribute: {attr.table_name}.{attr.attribute_name} Total Values: {len(attr.values)}")
                print(attr.position)

    return attributes
    
attributes = load_csv_files("/home/haseeb/Desktop/EKAI/ERD_automation/Dataset/train/northwind-db")            
            

Found 11 
 CSV files: ['employee_territories.csv', 'products.csv', 'orders.csv', 'customers.csv', 'territories.csv', 'orders_details.csv', 'suppliers.csv', 'employees.csv', 'categories.csv', 'shippers.csv', 'regions.csv']
Processing employee_territories.csv: 49 rows, 2 columns


TypeError: object of type 'int' has no len()

In [ ]:
def read_IND(file_path):
    '''
        fills up the dependent and reference array of attributes
    '''
    with open(file_path, "r") as f:
        for line in f:
            vars = line.strip().split("=")
            attributes[vars[0]].references = attributes[vars[1]]
            attributes[vars[1]].dependent = attributes[vars[0]]
read_IND("/home/haseeb/Desktop/EKAI/ERD_automation/codes/inclusionDependencyWithSpider/spider_results/northwind-db.txt")

KeyError: 'employee_territories.employeeid'

In [ ]:
for key, value in attributes.items():
    print(key + "->" + str(value.pkScore))

event.name->2.812766676853255
event.date->2.4162011610005836
event.type->2.4031579383843615
event.remark ->2.0685899250139372
pet.lexicon->3.0159583460665687
pet.owner->2.5039474229804517
pet.species->2.5039474229804517
pet.sex->2.3772145840653494
pet.birth->2.5159583460665687
pet.death->1.750981678752828
